# Telecom X - Análisis de Evasión de Clientes
Has sido contratado como asistente de análisis de datos en **Telecom X**. Este notebook contiene el Análisis Exploratorio de Datos (EDA) para entender los factores que influyen en la cancelación de clientes (`Churn`).

🔍 **Objetivo**: Explorar y visualizar los datos para identificar patrones, tendencias y relaciones útiles para modelos predictivos y estrategias de retención.

## 1. Carga de Librerías y Configuración Inicial

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
sns.set(style="whitegrid")

## 2. Extracción de Datos desde API (JSON remoto)

In [ ]:
# Descargar datos desde URL
url = "https://raw.githubusercontent.com/sthemonica/alura-voz/refs/heads/main/Dados/Telco-Customer-Churn.json"
response = requests.get(url)
data = response.json()
df = pd.json_normalize(data)

## 3. Transformación y Limpieza de Datos

In [ ]:
df.dropna(how="any", inplace=True)
df.reset_index(drop=True, inplace=True)
df['Churn'] = df['Churn'].fillna('No').replace('', 'No').str.strip()
df['account.Contract'] = df['account.Contract'].str.strip()
df['account.PaymentMethod'] = df['account.PaymentMethod'].str.strip()
df['customer.gender'] = df['customer.gender'].str.strip()
df['customer.tenure'] = pd.to_numeric(df['customer.tenure'], errors='coerce')
df.dropna(subset=['customer.tenure'], inplace=True)

## 4. Vista General de los Datos

In [ ]:
df.info()
df.head()

## 5. Análisis Exploratorio de Datos (EDA)

### 5.1 Cancelación por Tipo de Contrato

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x="account.Contract", hue="Churn", palette="Set2")
plt.title("Cancelación por tipo de contrato")
plt.xlabel("Tipo de contrato")
plt.ylabel("Cantidad de clientes")
plt.xticks(rotation=15)
plt.legend(title="¿Canceló?")
plt.tight_layout()
plt.show()

### 5.2 Cancelación por Forma de Pago

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x="account.PaymentMethod", hue="Churn", palette="Set2")
plt.title("Cancelación por forma de pago")
plt.xlabel("Forma de pago")
plt.ylabel("Cantidad de clientes")
plt.xticks(rotation=45)
plt.legend(title="¿Canceló?")
plt.tight_layout()
plt.show()

### 5.3 Tiempo de Contrato y Cancelación (Proporciones)

In [ ]:
df['tenure_group'] = pd.cut(df['customer.tenure'], bins=10)
tenure_df = df.groupby(['tenure_group', 'Churn']).size().reset_index(name='count')
total_per_group = tenure_df.groupby('tenure_group')['count'].transform('sum')
tenure_df['percentage'] = tenure_df['count'] / total_per_group * 100

plt.figure(figsize=(12, 6))
sns.barplot(data=tenure_df, x='tenure_group', y='percentage', hue='Churn', palette="Set2")
plt.title("Proporción de cancelaciones por tiempo de contrato")
plt.xlabel("Grupo de meses de contrato")
plt.ylabel("Porcentaje (%)")
plt.xticks(rotation=45)
plt.legend(title="¿Canceló?")
plt.tight_layout()
plt.show()

### 5.4 Cancelación por Género (Proporciones)

In [ ]:
gender_df = df.groupby(['customer.gender', 'Churn']).size().reset_index(name='count')
total_gender = gender_df.groupby('customer.gender')['count'].transform('sum')
gender_df['percentage'] = gender_df['count'] / total_gender * 100

plt.figure(figsize=(8, 5))
sns.barplot(data=gender_df, x='customer.gender', y='percentage', hue='Churn', palette="Set2")
plt.title("Proporción de cancelación por género")
plt.xlabel("Género")
plt.ylabel("Porcentaje (%)")
plt.legend(title="¿Canceló?")
plt.tight_layout()
plt.show()

## 6. Conclusiones y Próximos Pasos

- Los contratos mensuales presentan una mayor tasa de cancelación en comparación con los contratos de mayor duración.
- Los métodos de pago automáticos parecen estar asociados a menores tasas de cancelación.
- Los clientes con menos meses de permanencia tienen más probabilidad de cancelar.
- No se observan diferencias significativas entre géneros en cuanto a cancelación.

**Próximos pasos sugeridos:**
- Usar estas variables para entrenar un modelo de predicción de churn.
- Diseñar estrategias de fidelización según tipo de contrato o forma de pago.
- Realizar análisis adicional incluyendo otras variables como servicios contratados, ingresos, etc.